In [63]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import os
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
import torch

In [2]:
PATH = "./TrainingData/"
val_subject_number = 5



In [7]:
files = []
for filename in os.listdir(PATH):
    files.append(filename)
files = sorted(files, key = lambda x: (int(x.split('_')[1]),int(x.split('_')[2]), x.split('_')[4] ))


Calculating the value count of the window (15) and reducing the entries

In [58]:
def windows(d, w, t):
  r = np.arange(len(d))
  # print(r)
  s = r[::t]
  # print(s)
  z = list(zip(s, s + w))
  f = '{0[0]}:{0[1]}'.format
  g = lambda t: d.iloc[t[0]:t[1]]
  ranges = list(map(f,z))
  # print(ranges[-1])
  return ranges, pd.concat(map(g, z), keys=map(f, z))

Concatinating features and timestamp

In [104]:
X = []
Y = []
for i in range(0, len(files), 4):
    x_time, x, y_time, y = files[i: i + 4]
    x_time_df = pd.read_csv(PATH + x_time , header=None)
    x_df = pd.read_csv(PATH + x , header=None)
    x_combined = pd.concat([x_time_df, x_df], axis=1, ignore_index=True)
    x_combined = x_combined.loc[range(1,len(x_combined), 4)].reset_index()  # down sampled the frequency
    # print(x_combined.shape)

    y_time_df = pd.read_csv(PATH + y_time , header=None)
    y_df = pd.read_csv(PATH + y , header=None)
    y_combined = pd.concat([y_time_df, y_df], axis=1, ignore_index=True)
    # print(y_combined.shape)
    train_df = pd.concat([x_combined, y_combined], axis=1, ignore_index=True)
    train_df = train_df.drop(columns=[0, 1, 8])  # Dropping the time stamp
    Y.extend(train_df[9].values)
    print(train_df.drop(columns=[9]).values[0])
    X.extend(train_df.drop(columns=[9]).values[0])
    overlap = 15
    # print(train_df)
    # ranges, windows_df = windows(train_df, 30, overlap)
    # # print(windows_df.shape)
    # for ran in ranges:
    #     l,r = ran.split(':')
    #     df_range = windows_df.iloc[int(l): int(r)]
    #     if int(r) > len(windows_df):
    #         break
    #     y_values = df_range[9].values
    #     x_values = df_range.drop(columns=[9]).values
    #     print(x_values)
    #     X.append(x_values)
    #     # print(Counter(list(y_values)).most_common(1)[0][0])
    #     Y.append(Counter(list(y_values)).most_common(1)[0][0])
print(len(X), len(Y))

TypeError: 'numpy.int64' object is not iterable

The weights that compensate for the imbalance distributions

In [64]:
weight  = compute_class_weight(class_weight = 'balanced', classes = np.unique(Y), y = np.array(Y))
weight = torch.tensor(weight)

In [102]:
len(X)

29

In [97]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    stratify=Y, 
                                                    test_size=0.2)

d:\anaconda3\lib\site-packages\sklearn\utils\validation.py:746: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.asarray(array, order=order, dtype=dtype)


AttributeError: 'bool' object has no attribute 'any'

In [84]:
X_train = np.array(X_train)
y_train = np.array(y_train).reshape(-1,1)

In [71]:
Counter(y_train)

Counter({0: 13054, 3: 3478, 2: 448, 1: 920})

In [87]:
X_train[0]

array([[-2.615648  ,  8.477261  , 13.4497    ,  7.79267   , -0.2387123 ,
        -0.3467662 ],
       [-7.819409  ,  3.16739   ,  8.919365  , -3.38206   , -1.772744  ,
        -0.2648882 ],
       [-6.231173  ,  8.425428  , 14.94341   , -2.877805  ,  0.9527063 ,
         0.5746796 ],
       [-1.717818  ,  9.547422  ,  6.591871  , -1.330105  , -1.409305  ,
        -0.467174  ],
       [-2.111716  ,  9.098293  ,  3.350611  , -0.6741492 , -1.329205  ,
        -0.4670816 ],
       [-2.260614  ,  8.517343  ,  0.09903866, -1.072398  , -1.317475  ,
        -0.2679185 ],
       [-4.217178  ,  9.443285  , -1.412576  , -2.434406  , -0.4838025 ,
         0.3650663 ],
       [ 3.102272  , 16.31312   , -1.552971  , -4.180005  , -1.568144  ,
         0.7121762 ],
       [-4.024575  , 11.83685   , 17.43845   , -1.564761  ,  0.06709305,
         0.1678368 ],
       [-2.769921  ,  1.520739  ,  8.888709  ,  2.987751  , -0.2146547 ,
        -1.07917   ],
       [ 3.870735  ,  0.9826841 , 10.15286   ,  6.

In [78]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 500, max_depth = 4, max_features = 3, bootstrap = True, random_state = 18).fit(X_train, y_train)

ValueError: Found array with dim 3. Estimator expected <= 2.